# Assignment 2: AI-Powered Motivational Quote Generator

**Name:** Shakirth Anisha       
**SRN:** PES2UG23CS927        
**Section:** I

---

## Project Overview

### Category 6: Creative Writing & Content (Expansion on Generation)

#### Motivational Quote Generator
- **Goal**: Generate unique motivational quotes based on keywords like "Success", "Grind", "Peace".
- **Tech**: gpt2 with high temperature for creativity.

## Implementation

### Step 1: Environment Setup

In [14]:
# Install required packages (uncomment if needed)
# !pip install -q transformers torch

In [15]:
# Import necessary libraries
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
import torch
import random
import re
import warnings
warnings.filterwarnings('ignore')

device = 0 if torch.cuda.is_available() else -1
device_name = "GPU" if torch.cuda.is_available() else "CPU"

### Step 2: Load GPT-2 Model

In [16]:
# Load model
model_name = "gpt2"

try:
    quote_generator = pipeline(
        task="text-generation",
        model=model_name,
        device=device,
        pad_token_id=50256
    )

    print(f"Model Loaded: {model_name}")

except Exception as e:
    print(f"Error loading model: {e}")

Device set to use cuda:0


Model Loaded: gpt2


### Step 3: Core Quote Generation System

In [17]:
def generate_motivational_quote(
    theme: str,
    temperature: float = 1.0,
    num_quotes: int = 1,
    max_length: int = 60,
    style: str = "inspirational"
) -> list:

    style_prompts = {
        'inspirational': f"Inspirational quote about {theme}: \"",
        'tough-love': f"Motivational quote about {theme}: \"",
        'reflective': f"Thoughtful wisdom about {theme}: \"",
        'poetic': f"Poetic insight on {theme}: \""
    }

    prompt = style_prompts.get(style, style_prompts['inspirational'])

    try:
        generated = quote_generator(
            prompt,
            max_length=max_length,
            num_return_sequences=num_quotes,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )

        quotes = []
        for gen in generated:
            raw_quote = gen['generated_text']
            cleaned_quote = clean_quote(raw_quote, prompt)
            if cleaned_quote:
                quotes.append(cleaned_quote)

        return quotes if quotes else ["Error generating quote"]

    except Exception as e:
        return [f"Error generating quote: {str(e)}"]


def clean_quote(raw_text: str, prompt: str) -> str:
    quote = raw_text.replace(prompt, '').strip()
    sentences = re.split(r'[.!?]', quote)

    if sentences:
        quote = sentences[0].strip()
        quote = quote.rstrip('"').rstrip("'").strip()
        if len(quote.split()) >= 5:
            return f"\"{quote}.\""

    return None

### Step 4: Interactive Quote Display System

In [18]:
def display_quotes(theme: str, quotes: list, temperature: float, style: str):
    print("\n" + "═" * 80)
    print(f"THEME: {theme.upper()}")
    print(f"Style: {style.capitalize()}")
    print(f"Temperature: {temperature}")
    print("═" * 80)

    for i, quote in enumerate(quotes, 1):
        print(f"\nQuote #{i}:")
        print(f"   {quote}")

    print("\n" + "═" * 80 + "\n")


def batch_generate_quotes(themes: list, quotes_per_theme: int = 3):
    all_quotes = {}

    print(f"Generating {quotes_per_theme} quotes for {len(themes)} themes...\n")

    for theme in themes:
        print(f"   Processing theme: {theme}...")
        quotes = generate_motivational_quote(
            theme=theme,
            num_quotes=quotes_per_theme,
            temperature=1.0
        )
        all_quotes[theme] = quotes

    print(f"\nGenerated {len(themes) * quotes_per_theme} total quotes!\n")
    return all_quotes

## Demo & Experimentation

Now let's test our system with different themes and parameters!

### Example 1: Success & Achievement

In [19]:
theme = "Success"
style = "inspirational"
temperature = 0.9

print("Generating Success quotes...\n")

success_quotes = generate_motivational_quote(
    theme=theme,
    temperature=temperature,
    num_quotes=3,
    style=style
)

display_quotes(theme, success_quotes, temperature, style)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generating Success quotes...


════════════════════════════════════════════════════════════════════════════════
THEME: SUCCESS
Style: Inspirational
Temperature: 0.9
════════════════════════════════════════════════════════════════════════════════

Quote #1:
   "There is no such thing as a failure, I am just too busy."

════════════════════════════════════════════════════════════════════════════════



### Example 2: Grind & Perseverance (Tough-Love Style)

In [20]:
theme = "Grind"
style = "tough-love"
temperature = 1.1  # Higher temperature for more aggressive quotes

grind_quotes = generate_motivational_quote(
    theme=theme,
    temperature=temperature,
    num_quotes=3,
    style=style
)

display_quotes(theme, grind_quotes, temperature, style)

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



════════════════════════════════════════════════════════════════════════════════
THEME: GRIND
Style: Tough-love
Temperature: 1.1
════════════════════════════════════════════════════════════════════════════════

Quote #1:
   "If you can get out of bed, do it right."

Quote #2:
   "I've spent the last three years working in development and planning my first feature release — it's called 'The Holographic Guide to Coder Development."

Quote #3:
   "To become an agent, you have to prove yourself before the agency or on your own."

════════════════════════════════════════════════════════════════════════════════



### Example 3: Peace & Mindfulness (Reflective Style)

In [21]:
theme = "Peace"
style = "reflective"
temperature = 0.85  # Slightly lower for calm quotes

peace_quotes = generate_motivational_quote(
    theme=theme,
    temperature=temperature,
    num_quotes=3,
    style=style
)

display_quotes(theme, peace_quotes, temperature, style)

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



════════════════════════════════════════════════════════════════════════════════
THEME: PEACE
Style: Reflective
Temperature: 0.85
════════════════════════════════════════════════════════════════════════════════

Quote #1:
   "There are two kinds of peace in heaven and earth."

Quote #2:
   "The Church is the only one of us who will ever truly know all that it teaches and believes."

Quote #3:
   "In God we believe and in the Covenant, He has promised to bring peace through His love for all mankind."

════════════════════════════════════════════════════════════════════════════════



### Example 4: Innovation & Creativity

In [22]:
theme = "Innovation"
style = "poetic"
temperature = 1.2  # tis high for max creativity

print("Generating Innovation quotes (poetic style)...\n")

innovation_quotes = generate_motivational_quote(
    theme=theme,
    temperature=temperature,
    num_quotes=3,
    style=style
)

display_quotes(theme, innovation_quotes, temperature, style)

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generating Innovation quotes (poetic style)...


════════════════════════════════════════════════════════════════════════════════
THEME: INNOVATION
Style: Poetic
Temperature: 1.2
════════════════════════════════════════════════════════════════════════════════

Quote #1:
   "One of the great pleasures in studying history, particularly during World War II and Korea is witnessing what it takes to advance our understanding… The most promising aspect that all those years later should look forward towards as they prepare to enter a new field with regard not only for success but even its future."

Quote #2:
   "The new technologies and innovations coming into this marketplace may take place across multiple domains (digital assets, infrastructure networks and businesses)."

Quote #3:
   "We have been making a big deal out of our work, having done more research and development for the iPad."

════════════════════════════════════════════════════════════════════════════════



### Example 5: Goals & Ambition

In [23]:
theme = "Goals"
style = "inspirational"
temperature = 1.0  # Balanced ig (I'm just spamming it at this point)

print("Generating Goals quotes...\n")

goals_quotes = generate_motivational_quote(
    theme=theme,
    temperature=temperature,
    num_quotes=3,
    style=style
)

display_quotes(theme, goals_quotes, temperature, style)

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generating Goals quotes...


════════════════════════════════════════════════════════════════════════════════
THEME: GOALS
Style: Inspirational
Temperature: 1.0
════════════════════════════════════════════════════════════════════════════════

Quote #1:
   "We are the good guys."

Quote #2:
   "When they come, the goalkeepers go and have fun watching them play."

Quote #3:
   "We have a new game coming up which makes you better as players."

════════════════════════════════════════════════════════════════════════════════



## Example 6 - Low Temp and Courage

In [24]:
low_temp_quotes = generate_motivational_quote(
    theme="Courage",
    temperature=0.7,
    num_quotes=2
)

display_quotes("Courage", low_temp_quotes, 0.7, "inspirational")

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



════════════════════════════════════════════════════════════════════════════════
THEME: COURAGE
Style: Inspirational
Temperature: 0.7
════════════════════════════════════════════════════════════════════════════════

Quote #1:
   "It is not a matter of whether or to what extent you believe in something, but rather how much belief can be justified."

Quote #2:
   "There is no way you could get to that point without a lot of thinking, but there are ways we can improve our lives."

════════════════════════════════════════════════════════════════════════════════



### Example 7: High Temperature and Courage

In [25]:
high_temp_quotes = generate_motivational_quote(
    theme="Courage",
    temperature=1.3,
    num_quotes=2
)

display_quotes("Courage", high_temp_quotes, 1.3, "inspirational")

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



════════════════════════════════════════════════════════════════════════════════
THEME: COURAGE
Style: Inspirational
Temperature: 1.3
════════════════════════════════════════════════════════════════════════════════

Quote #1:
   "You're not even a hero because you are an unshakable evil," said the speaker, to tears."

Quote #2:
   "This past week I gave her to you."

════════════════════════════════════════════════════════════════════════════════



## Batch Generation for Content Creation

Perfect for social media managers or content creators!

In [26]:
content_themes = [
    "Success",
    "Grind",
    "Peace",
    "Dreams",
    "Resilience"
]

batch_results = batch_generate_quotes(content_themes, quotes_per_theme=2)

print("RESULTS")
print("=" * 80)

for theme, quotes in batch_results.items():
    print(f"\n{theme.upper()}:")
    for i, quote in enumerate(quotes, 1):
        print(f"  {i}. {quote}")

print("\n" + "=" * 80)

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generating 2 quotes for 5 themes...

   Processing theme: Success...


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Processing theme: Grind...


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Processing theme: Peace...


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Processing theme: Dreams...


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Processing theme: Resilience...

Generated 10 total quotes!

RESULTS

SUCCESS:
  1. "Our lives are important because if we can get to the things that make us great, then our entire group will be perfect."
  2. "Our bodies are still our friends."

GRIND:
  1. "I'm a big fan of their 'Nexus' tour."
  2. "The point is if you do it and don't care how long, the result can be just what we have been wanting for."

PEACE:
  1. "When you look at the world, it's filled with things that no one can understand."
  2. "It is not your enemy."

DREAMS:
  1. "A dream is a kind of knowledge that you do not know."
  2. "A person who dreams that one day he or she will actually see something to believe in" is not a dreamer."

RESILIENCE:
  1. "Everything you learn in life makes up for it, not just how much your brain can handle."
  2. "One can imagine a time when you are not at your strongest."



# Summary

## Key Parameters

**Temperature:**
- 0.7: Focused, predictable, safer
- 1.0: Balanced creativity (recommended)
- 1.3: Very creative, more diverse (may be less coherent)

**Top-k (50):**
- Considers top 50 most likely next tokens
- Balances diversity and quality

**Top-p (0.95):**
- Nucleus sampling
- Selects from smallest set of tokens with cumulative probability 95%
- Prevents very unlikely tokens

**Repetition Penalty (1.2):**
- Discourages repeated phrases
- Ensures variety in quotes

## Limitations

- May occasionally generate generic quotes
- Quality varies with temperature settings
- No guarantee of factual accuracy
- Limited to GPT-2's training knowledge



## Conclusion

This project shows the application of genAI for creative content creation. It uses GPT-2's language generation with custom parametersand generated relevant motivational quotes on demand.

---
